# Basic data processing

In [1]:
import os
import spacepy

ModuleNotFoundError: No module named 'spacepy'

## Loading data from ftp server
`TODO` Write a bash/python script to load data from the ftp server automatically.   
Currently the notebook requires that you download the data manually. 

## Extracting `.tar.gz` files

In [9]:
# Extracts all tar files from data/ directory
!for f in ../data/*.tar.gz; do tar -xvf "$f" -C ../data/; done;

juicepsa-pds4-PI-01-juice_rad-20230416T180019/juice_rad/data_raw/rad_raw_sc_20230416.lblx
juicepsa-pds4-PI-01-juice_rad-20230416T180019/juice_rad/data_raw/rad_raw_sc_20230416.cdf
juicepsa-pds4-PI-01-juice_rad-20230416T180019/juicepsa-pds4-PI-01-juice_rad-20230416T180019-checksum_manifest.tab
juicepsa-pds4-PI-01-juice_rad-20230416T180019/juicepsa-pds4-PI-01-juice_rad-20230416T180019-transfer_manifest.tab
juicepsa-pds4-PI-01-juice_rad-20230416T180019/juicepsa-pds4-PI-01-juice_rad-20230416T180019.xml
juicepsa-pds4-PI-01-juice_rad-20230419T213312/juice_rad/data_raw/rad_raw_sc_20230418.lblx
juicepsa-pds4-PI-01-juice_rad-20230419T213312/juice_rad/data_raw/rad_raw_sc_20230418.cdf
juicepsa-pds4-PI-01-juice_rad-20230419T213312/juicepsa-pds4-PI-01-juice_rad-20230419T213312-checksum_manifest.tab
juicepsa-pds4-PI-01-juice_rad-20230419T213312/juicepsa-pds4-PI-01-juice_rad-20230419T213312-transfer_manifest.tab
juicepsa-pds4-PI-01-juice_rad-20230419T213312/juicepsa-pds4-PI-01-juice_rad-20230419T21331

In [10]:
# Remove tar.gz files
!rm -rf ../data/*.tar.gz

In [11]:
for file in os.listdir("../data/"):
    print(os.path.join("../data/", file))

../data/.gitkeep
../data/juicepsa-pds4-PI-01-juice_rad-20230416T180019
../data/juicepsa-pds4-PI-01-juice_rad-20230419T213312
../data/juicepsa-pds4-PI-01-juice_rad-20230419T213444
../data/juicepsa-pds4-PI-01-juice_rad-20230421T000051
../data/juicepsa-pds4-PI-01-juice_rad-20230421T000226
../data/juicepsa-pds4-PI-01-juice_rad-20230425T092621
../data/juicepsa-pds4-PI-01-juice_rad-20230426T084435
../data/juicepsa-pds4-PI-01-juice_rad-20230426T084607
../data/juicepsa-pds4-PI-01-juice_rad-20230707T185430
../data/juicepsa-pds4-PI-01-juice_rad-20230709T185431
../data/juicepsa-pds4-PI-01-juice_rad-20230720T153618
../data/juicepsa-pds4-PI-01-juice_rad-20230901T105022
../data/juicepsa-pds4-PI-01-juice_rad-20230912T144122
../data/juicepsa-pds4-PI-01-juice_rad-20230912T144248
../data/juicepsa-pds4-PI-01-juice_rad-20230912T144414
../data/juicepsa-pds4-PI-01-juice_rad-20230912T144541
../data/juicepsa-pds4-PI-01-juice_rad-20230912T144708
../data/juicepsa-pds4-PI-01-juice_rad-20230912T144834
../data/jui